# A Test for the Cytoscape Example

In [1]:
import json
import os
import time
from pathlib import Path

import importnb
from rdflib import Graph
from requests_cache import CachedSession

with importnb.Notebook():
    try:
        from Cytoscape_Example import cyto_ex, W
    except ImportError:
        from .Cytoscape_Example import cyto_ex, W

This notebook is used as part of the `ipyradiant` test suite, where `IPYRADIANT_TESTING`
will be set, which will trigger the automted section below.

In [2]:
IS_TESTING = json.loads(os.environ.get("IPYRADIANT_TESTING", "false"))
IS_TESTING

False

Example data is loaded from [schema.org](https://schema.org/docs/developers.html).

In [4]:
TEST_URL = "https://raw.githubusercontent.com/schemaorg/schemaorg/v9.0-release/data/releases/8.0/schema.{fmt}"

Tests are implemented as buttons you click.

In [5]:
tests = [
    W.Button(description=f"{fmt}") for fmt in ["ttl", "nt", "rdf"]
]
cyto_ex.layout.width = "100%"

In [6]:
timings = {}


def timestamp(key, msg):
    if key not in timings:
        timings[key] = []
    timings[key] += [time.time()]
    delta = 0 if len(timings[key]) == 1 else timings[key][-1] - timings[key][-2]
    print(f"[{key}]", f"+{int(delta)}", msg)

In [7]:
def _run_test(btn):
    session = CachedSession(".cache")
    fmt = btn.description

    try:
        timestamp(fmt, "starting...")
        cyto_ex.loader.graph = Graph()
        timestamp(fmt, "cleaned...")
        assert not cyto_ex.cyto_widget.graph.edges
        assert not cyto_ex.cyto_widget.graph.nodes
        url = TEST_URL.format(fmt=fmt)
        timestamp(fmt, f"fetching {url}...")
        response = session.get(url)
        timestamp(fmt, f"... response: {response.status_code}")
        assert response.status_code == 200
        timestamp(fmt, f"... {len(response.text)} bytes")
        timestamp(fmt, f"loading...")
        cyto_ex.loader.load_box.file_upload_value = {
            Path(url).name: {
                "metadata": {"length": len(response.text)},
                "content": response.text,
            }
        }
        timestamp(fmt, f"... {len(cyto_ex.loader.graph)} triples loaded")
        assert len(cyto_ex.loader.graph)
        assert cyto_ex.cyto_widget.graph.edges
        assert cyto_ex.cyto_widget.graph.nodes
        timestamp(fmt, "OK!")
    except Exception as err:
        timestamp(fmt, "ERROR")
        timestamp(fmt, err)
        raise Exception("fmt failed") from err


[d.on_click(_run_test) for d in tests]

[None, None, None]

## Show the Test Application

In [9]:
W.VBox([cyto_ex, W.HBox([W.Label("Start Test"), *tests])])

# Run Test Automatically

In [11]:
if IS_TESTING:
    for test in tests:
        test.click()